In [28]:
import reconstruccion3D as r3D
import numpy as np
import matplotlib.pyplot as plot
import cv2

In [ ]:
%matplotlib inline

In [29]:
ruta = "img/monumento/"
K = np.array([[2759.48/4, 0, 1520.69/4, 0, 2764.16/4,1006.81/4, 0, 0, 1]]).reshape(3, 3)
d = np.array([0.0, 0.0, 0.0, 0.0, 0.0]).reshape(1, 5)
a = r3D.Rescontruccion3D(ruta,K,d)

In [30]:
a.inicializacion()

In [31]:
a.salvar_nube("monumento.pcd")

In [32]:
a.graficar_nube()

In [33]:
vista_uno = a.vistas[0]
vista_dos = a.vistas[2]
vista_uno.buscar_feature(vista_dos,a.matcher)

In [34]:
#incorporando otras vistas.
new_feature = vista_uno.features[vista_dos]
old_feature = vista_uno.features[a.vistas[1]]

In [35]:
match_utilizados =[]
j = 0
for i in range(a.FMask.shape[0]):
    if a.FMask[i]:
        punto3D = a.pts3D[j]
        match_utilizados.append((old_feature.matches[i],punto3D))
        j = j+1

In [36]:
match_c = []
puntos_3D = []
for old_match,punto3D in match_utilizados:
    for new_match in new_feature.matches:
        if new_match.queryIdx == old_match.queryIdx:
            match_c.append(old_match)
            puntos_3D.append(punto3D)

In [37]:
match_c.__len__()
puntos_3D.__len__()

143

In [38]:
puntos_2D = []
for new_match in match_c:
    #print new_feature.puntos_dos[new_match.trainIdx].pt
    puntos_2D.append(new_feature.puntos_dos[new_match.trainIdx].pt)

In [39]:
retval,mask=cv2.findFundamentalMat(new_feature.match_puntos_uno,new_feature.match_puntos_dos,cv2.FM_RANSAC, 0.1, 0.99)

In [54]:
puntos_2D = np.array(puntos_2D)
puntos_3D = np.array(puntos_3D)
rvec,t,p = cv2.solvePnP(puntos_3D, puntos_2D, a.K,None)
R,j = cv2.Rodrigues(t,a.R)
P3 = np.hstack((R, a.t.reshape(3, 1)))
ptsH1,ptsH2 = new_feature.homogeneizar_puntos(a.K,mask)
X = a.triangular(ptsH1,ptsH2,P3)

In [56]:
r = np.concatenate((X,a.pts3D))
r.shape
a.pts3D = r 
a.pts3D.shape

(279, 3)

In [ ]:
a.graficar_nube()